In [14]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [15]:
current_directory = Path.cwd()
relative_path = Path('order_sales_data.csv')
absolute_path = current_directory / relative_path


df = pd.read_csv(absolute_path)
df.head()

/tmp/ipykernel_3047/3307893245.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(absolute_path)


,order_id,order_data,store_name,product_category_name,product_subcategory_name,product_name,order_qty,state_province_name,country_region_name,territory_name
0,59005,2013-10-30,Summer Sports Place,Components,Saddles,LL Mountain Seat/Saddle,1,Illinois,United States,Central
1,51796,2013-06-30,Bike Products and Accessories,Accessories,Tires and Tubes,Patch Kit/8 Patches,3,Virginia,United States,Southeast
2,51796,2013-06-30,Bike Products and Accessories,Components,Saddles,LL Mountain Seat/Saddle,1,Virginia,United States,Southeast
3,47393,2012-07-31,Summer Sports Place,Components,Headsets,LL Headset,1,Illinois,United States,Central
4,47985,2012-09-30,Security Racks and Locks Wholesalers,Components,Headsets,LL Headset,1,Nevada,United States,Northwest
